In [50]:
import numpy as np
import pandas as pd
import math
import random
import matplotlib.pyplot as plt
from scipy.stats import beta

In [51]:
##Assume R_EXPR_Bucket3 and R_EXPT_Bucket1 came up as two best buckets 
##with consistent high rewards after 100 iterations

##1.Exploit Only Approach: samples are only picked up from exploit buckets
##2.Explore and Exploit at Random: samples are randomly picked from explore and exploit buckets
##3.Concurrent Arm Beta Sampling: samples are picked from the buckets based on the recent rewards.


In [52]:
df_AL_prog = pd.read_excel("AL_data1.xlsx")

In [53]:
df_AL_prog

,Day,R_EXPR_Bucket1,R_EXPR_Bucket2,R_EXPR_Bucket3,R_EXPR_Bucket4,R_EXPT_Bucket1,R_EXPT_Bucket2,R_EXPT_Bucket3,R_EXPT_Bucket4,SR_EXPR_Bucket1,SR_EXPR_Bucket2,SR_EXPR_Bucket3,SR_EXPR_Bucket4,SR_EXPT_Bucket1,SR_EXPT_Bucket2,SR_EXPT_Bucket3,SR_EXPT_Bucket4
0,1,0.497405,0.031951,0.061860,0.039905,0.148002,0.147913,0.011875,0.033545,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,2,0.541126,0.081003,0.107044,0.068148,0.120225,0.060056,0.139287,0.072630,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2,3,0.684797,0.030485,0.076257,0.093349,0.051888,0.037260,0.058451,0.022623,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
3,4,0.076572,0.059089,0.040310,0.036148,0.053363,0.125806,0.068863,0.024508,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,5,0.223486,0.036944,0.056904,0.049597,0.054044,0.055858,0.090199,0.088001,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
5,6,0.037236,0.028764,0.073113,0.135859,0.029296,0.075040,0.148648,0.077736,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
6,7,0.253326,0.055542,0.032859,0.010922,0.035297,0.131125,0.126713,0.106387,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
7,8,0.105901,0.012873,0.034029,0.047224,0.015673,0.057894,0.091646,0.029130,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
8,9,0.283862,0.081162,0.111623,0.077575,0.015753,0.116151,0.113592,0.068817,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
9,10,0.566960,0.095745,0.071987,0.091604,0.090816,0.033389,0.000307,0.022791,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [54]:
df_AL_prog.columns

Index(['Day', 'R_EXPR_Bucket1', 'R_EXPR_Bucket2', 'R_EXPR_Bucket3',
       'R_EXPR_Bucket4', 'R_EXPT_Bucket1', 'R_EXPT_Bucket2', 'R_EXPT_Bucket3',
       'R_EXPT_Bucket4', 'SR_EXPR_Bucket1', 'SR_EXPR_Bucket2',
       'SR_EXPR_Bucket3', 'SR_EXPR_Bucket4', 'SR_EXPT_Bucket1',
       'SR_EXPT_Bucket2', 'SR_EXPT_Bucket3', 'SR_EXPT_Bucket4'],
      dtype='object')

In [55]:
###1. Only Exploitation
#####Exploit Only Approach: samples are only picked up from exploit buckets

iter = 0

for iter in range(100):
    
    #print(iter)
    if iter == 0 :
        df_AL_prog = pd.read_excel("AL_data1.xlsx")



    max_tot_samples = 15

    curr_day = df_AL_prog[df_AL_prog['R_EXPT_Bucket1'].isnull()].iloc[0,0]


    df_rewards = df_AL_prog[[col for col in df_AL_prog if col.startswith('R')]]


    df_rewards[df_rewards.columns].mean()

    df_SR_cols = [col for col in df_AL_prog if col.startswith('SR')]

    df_team_cols = [col for col in df_AL_prog if col.startswith('R')]


    no_groups = len(df_SR_cols)



    x = np.linspace(0, 1, 1002)[1:-1]
#     x

    ####Get the prior values of a and b based on rewards from last few days and then update 
    ###the sampling ratio for all  buckets by comparing with the previous days sampling ratio  

    for i,j in zip(df_team_cols,df_SR_cols):
        #print(i)
        #print(j)
        if (i.startswith('R_EXPT')):
            a = 8 + random.random()
            b = random.random()
        else:
            a = random.random()
            b = 8 + random.random()
            

        ###Get a random sample for sampling ratio from the beta distribution for the current team 
        ###based on last 3 weeks rewards
        random_beta = np.random.beta(a, b)

        ###Get the last week's sampling ratio for the team 
        max_beta = (df_AL_prog[~df_AL_prog[i].isnull()].tail(3)[i].mean() ) / max_tot_samples


        df_AL_prog[j].iloc[curr_day - 1] = random_beta

    sum_SR = 0
    for col in df_SR_cols:

        sum_SR = sum_SR + df_AL_prog[col].iloc[curr_day - 1]

    for col in df_SR_cols:
        if sum_SR  != 0:
            df_AL_prog[col].iloc[curr_day - 1] = math.floor( (df_AL_prog[col].iloc[curr_day - 1] / sum_SR)* max_tot_samples)



    ####Randomly generate the rewards for next day
    for i in df_team_cols:
        if i == 'R_EXPT_Bucket1':
            df_AL_prog[i].iloc[curr_day - 1] = (0.7 + (random.random()* 0.2 ) )
        elif  i == 'R_EXPR_Bucket3':
            df_AL_prog[i].iloc[curr_day - 1] = (0.7 + (random.random()* 0.2 ) )
        else:
            df_AL_prog[i].iloc[curr_day - 1] = (0 + (random.random()* 0.4 ) )

In [56]:
df_AL_prog.head(100)

,Day,R_EXPR_Bucket1,R_EXPR_Bucket2,R_EXPR_Bucket3,R_EXPR_Bucket4,R_EXPT_Bucket1,R_EXPT_Bucket2,R_EXPT_Bucket3,R_EXPT_Bucket4,SR_EXPR_Bucket1,SR_EXPR_Bucket2,SR_EXPR_Bucket3,SR_EXPR_Bucket4,SR_EXPT_Bucket1,SR_EXPT_Bucket2,SR_EXPT_Bucket3,SR_EXPT_Bucket4
0,1,0.497405,0.031951,0.061860,0.039905,0.148002,0.147913,0.011875,0.033545,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,2,0.541126,0.081003,0.107044,0.068148,0.120225,0.060056,0.139287,0.072630,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2,3,0.684797,0.030485,0.076257,0.093349,0.051888,0.037260,0.058451,0.022623,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
3,4,0.076572,0.059089,0.040310,0.036148,0.053363,0.125806,0.068863,0.024508,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,5,0.223486,0.036944,0.056904,0.049597,0.054044,0.055858,0.090199,0.088001,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
5,6,0.037236,0.028764,0.073113,0.135859,0.029296,0.075040,0.148648,0.077736,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
6,7,0.253326,0.055542,0.032859,0.010922,0.035297,0.131125,0.126713,0.106387,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
7,8,0.105901,0.012873,0.034029,0.047224,0.015673,0.057894,0.091646,0.029130,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
8,9,0.283862,0.081162,0.111623,0.077575,0.015753,0.116151,0.113592,0.068817,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
9,10,0.566960,0.095745,0.071987,0.091604,0.090816,0.033389,0.000307,0.022791,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [57]:
###2. Explore and Exploit at Random
###If sampling is done at random

iter = 0

for iter in range(100):
    
    #print(iter)
    if iter == 0 :
        df_AL_prog = pd.read_excel("AL_data1.xlsx")



    max_tot_samples = 15

    curr_day = df_AL_prog[df_AL_prog['R_EXPT_Bucket1'].isnull()].iloc[0,0]


    df_rewards = df_AL_prog[[col for col in df_AL_prog if col.startswith('R')]]

    ####Incremental Benefit till now
    df_rewards[df_rewards.columns].mean()

    df_SR_cols = [col for col in df_AL_prog if col.startswith('SR')]

    df_team_cols = [col for col in df_AL_prog if col.startswith('R')]


    no_groups = len(df_SR_cols)

    ##Generate Sample data for tracking Beta distribution for different buckets
    x = np.linspace(0, 1, 1002)[1:-1]


    ####Get the prior values of a and b based on rewards from last few days and then update 
    ###the sampling ratio for all  teams by comparing with the previous days sampling ratio  

    for i,j in zip(df_team_cols,df_SR_cols):

        a = random.random()*10
        b = random.random()*10

        ###Get a random sample for sampling ratio from the beta distribution for the current team 
        ###based on last 3 weeks rewards
        random_beta = np.random.beta(a, b)

        ###Get the last week's sampling ratio for the team 
        max_beta = (df_AL_prog[~df_AL_prog[i].isnull()].tail(3)[i].mean() ) / max_tot_samples


        ###Updating the sampling ratios of the buckets for the next day
  
        df_AL_prog[j].iloc[curr_day - 1] = random_beta

    sum_SR = 0
    for col in df_SR_cols:

        sum_SR = sum_SR + df_AL_prog[col].iloc[curr_day - 1]

    for col in df_SR_cols:
        if sum_SR  != 0:
            df_AL_prog[col].iloc[curr_day - 1] = math.floor( (df_AL_prog[col].iloc[curr_day - 1] / sum_SR)* max_tot_samples)



    ####Randomly generate the rewards for next week. In the application, we will get from the user feedback table
    for i in df_team_cols:
        if i == 'R_EXPT_Bucket1':
            df_AL_prog[i].iloc[curr_day - 1] = (0.7 + (random.random()* 0.2 ) )
        elif  i == 'R_EXPR_Bucket3':
            df_AL_prog[i].iloc[curr_day - 1] = (0.7 + (random.random()* 0.2 ) )
        else:
            df_AL_prog[i].iloc[curr_day - 1] = (0 + (random.random()* 0.4 ) )

In [58]:
df_AL_prog.head(100)

,Day,R_EXPR_Bucket1,R_EXPR_Bucket2,R_EXPR_Bucket3,R_EXPR_Bucket4,R_EXPT_Bucket1,R_EXPT_Bucket2,R_EXPT_Bucket3,R_EXPT_Bucket4,SR_EXPR_Bucket1,SR_EXPR_Bucket2,SR_EXPR_Bucket3,SR_EXPR_Bucket4,SR_EXPT_Bucket1,SR_EXPT_Bucket2,SR_EXPT_Bucket3,SR_EXPT_Bucket4
0,1,0.497405,0.031951,0.061860,0.039905,0.148002,0.147913,0.011875,0.033545,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,2,0.541126,0.081003,0.107044,0.068148,0.120225,0.060056,0.139287,0.072630,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2,3,0.684797,0.030485,0.076257,0.093349,0.051888,0.037260,0.058451,0.022623,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
3,4,0.076572,0.059089,0.040310,0.036148,0.053363,0.125806,0.068863,0.024508,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,5,0.223486,0.036944,0.056904,0.049597,0.054044,0.055858,0.090199,0.088001,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
5,6,0.037236,0.028764,0.073113,0.135859,0.029296,0.075040,0.148648,0.077736,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
6,7,0.253326,0.055542,0.032859,0.010922,0.035297,0.131125,0.126713,0.106387,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
7,8,0.105901,0.012873,0.034029,0.047224,0.015673,0.057894,0.091646,0.029130,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
8,9,0.283862,0.081162,0.111623,0.077575,0.015753,0.116151,0.113592,0.068817,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
9,10,0.566960,0.095745,0.071987,0.091604,0.090816,0.033389,0.000307,0.022791,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [59]:
###3. Concurrent Arm Beta Sampling
iter = 0

for iter in range(100):
    
    #print(iter)
    if iter == 0 :
        df_AL_prog = pd.read_excel("AL_data1.xlsx")



    max_tot_samples = 15

    curr_day = df_AL_prog[df_AL_prog['R_EXPT_Bucket1'].isnull()].iloc[0,0]


    df_rewards = df_AL_prog[[col for col in df_AL_prog if col.startswith('R')]]

    ####Incremental Benefit till now
    df_rewards[df_rewards.columns].mean()

    df_SR_cols = [col for col in df_AL_prog if col.startswith('SR')]

    df_team_cols = [col for col in df_AL_prog if col.startswith('R')]



    no_groups = len(df_SR_cols)


    ##Generate Sample data for tracking Beta distribution for different buckets
    x = np.linspace(0, 1, 1002)[1:-1]


    ####Get the prior values of a and b based on rewards from last few days and then update 
    ###the sampling ratio for all  buckets by comparing with the previous day's sampling ratio  
    ###If sampling ratio < 
    for i,j in zip(df_team_cols,df_SR_cols):
 
        a = df_AL_prog[i].mean() * 10
        b = (1 - df_AL_prog[i].mean()) * 10 
        #print('a = '+ str(a))
        #print('b = '+ str(b))
        #print('\n')

#         ####Visulaize the beta distribution for the current team based on last 3 weeks rewards
#         dist = beta(a, b)
#         plt.plot(x, dist.pdf(x), c='black',label=r'$\alpha=%.1f,\ \beta=%.1f$' % (a, b))
#         plt.xlim(0, 1)
#         plt.ylim(0, 3)
#         plt.xlabel('$x$')
#         plt.ylabel(r'$p(x|\alpha,\beta)$')
#         plt.title('Beta Distribution')
#         plt.legend(loc=0)
#         plt.show()

        ###Get a random sample for sampling ratio from the beta distribution for the current bucket 
        ###based on last 3 days rewards
        random_beta = np.random.beta(a, b)

        ###Get the last week's sampling ratio for the team 
        max_beta = (df_AL_prog[~df_AL_prog[i].isnull()].tail(3)[i].mean() ) / max_tot_samples
#         print('max_sr '+ str(max_beta))
#         if random_beta > max_beta:
# #             print('Beta Updated')
#             max_beta = random_beta #update the maximum beta value thus far observed
#             print('max_sr_updated '+ str(max_beta) )
#             print('\n')
        max_beta = random_beta

        ###Updating the sampling ratios of the teams for the next day
        df_AL_prog[j].iloc[curr_day - 1] = max_beta

    sum_SR = 0
    for col in df_SR_cols:

        sum_SR = sum_SR + df_AL_prog[col].iloc[curr_day - 1]
#         print('sum_sr  ' + str(sum_SR) )
    for col in df_SR_cols:
        if sum_SR  != 0:
            df_AL_prog[col].iloc[curr_day - 1] = math.ceil( (df_AL_prog[col].iloc[curr_day - 1] / sum_SR)* max_tot_samples)



    ####Randomly generate the rewards for next day
    for i in df_team_cols:
        if i == 'R_EXPT_Bucket1':
            df_AL_prog[i].iloc[curr_day - 1] = (0.7 + (random.random()* 0.2 ) )
        elif  i == 'R_EXPR_Bucket3':
            df_AL_prog[i].iloc[curr_day - 1] = (0.7 + (random.random()* 0.2 ) )
        else:
            df_AL_prog[i].iloc[curr_day - 1] = (0 + (random.random()* 0.4 ) )

In [60]:
df_AL_prog.head(100)

,Day,R_EXPR_Bucket1,R_EXPR_Bucket2,R_EXPR_Bucket3,R_EXPR_Bucket4,R_EXPT_Bucket1,R_EXPT_Bucket2,R_EXPT_Bucket3,R_EXPT_Bucket4,SR_EXPR_Bucket1,SR_EXPR_Bucket2,SR_EXPR_Bucket3,SR_EXPR_Bucket4,SR_EXPT_Bucket1,SR_EXPT_Bucket2,SR_EXPT_Bucket3,SR_EXPT_Bucket4
0,1,0.497405,0.031951,0.061860,0.039905,0.148002,0.147913,0.011875,0.033545,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,2,0.541126,0.081003,0.107044,0.068148,0.120225,0.060056,0.139287,0.072630,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2,3,0.684797,0.030485,0.076257,0.093349,0.051888,0.037260,0.058451,0.022623,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
3,4,0.076572,0.059089,0.040310,0.036148,0.053363,0.125806,0.068863,0.024508,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,5,0.223486,0.036944,0.056904,0.049597,0.054044,0.055858,0.090199,0.088001,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
5,6,0.037236,0.028764,0.073113,0.135859,0.029296,0.075040,0.148648,0.077736,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
6,7,0.253326,0.055542,0.032859,0.010922,0.035297,0.131125,0.126713,0.106387,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
7,8,0.105901,0.012873,0.034029,0.047224,0.015673,0.057894,0.091646,0.029130,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
8,9,0.283862,0.081162,0.111623,0.077575,0.015753,0.116151,0.113592,0.068817,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
9,10,0.566960,0.095745,0.071987,0.091604,0.090816,0.033389,0.000307,0.022791,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
